In [2]:
!pip install -q pyspark==3.3.0 spark-nlp==4.2.1

In [3]:
import sparknlp
import pandas as pd

spark = sparknlp.start()


from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType, IntegerType

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

:: loading settings :: url = jar:file:/opt/conda/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-052da9c5-2640-4cd0-b617-d574a375fba4;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in 

22/11/27 08:27:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.1
Apache Spark version: 3.3.0


In [17]:
df = spark.read.csv("/kaggle/input/cord19truncated/parsed_papers_1k.csv",
                    inferSchema=True, header=True,
                    multiLine=True, quote='\"', escape='\"')
df.printSchema()

root
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- body_text: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- journal: string (nullable = true)



In [18]:
df = df.drop(*["authors", "abstract", "publish_time", "journal"])

df.show(5)

+--------+--------------------+--------------------+
|paper_id|               title|           body_text|
+--------+--------------------+--------------------+
|PMC35282|Clinical features...|Mycoplasma pneumo...|
|PMC59543|Nitric oxide: a p...|Since its discove...|
|PMC59549|Surfactant protei...|Surfactant protei...|
|PMC59574|Role of endotheli...|ET-1, ET-2, and E...|
|PMC59580|Gene expression i...|RSV and PVM are v...|
+--------+--------------------+--------------------+
only showing top 5 rows



In [7]:
documentAssembler = MultiDocumentAssembler() \
        .setInputCols(["question", "context"]) \
        .setOutputCols(["document_question", "document_context"])

In [12]:
spanClassifier = RoBertaForQuestionAnswering.pretrained("roberta_qa_roberta_base_squad2_covid","en")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer") \
    .setCaseSensitive(True)

roberta_qa_roberta_base_squad2_covid download started this may take some time.
Approximate size to download 442.8 MB
[OK!]


In [13]:
pipeline = Pipeline().setStages([
    documentAssembler,
    spanClassifier])

In [47]:
# sample_texts = [
#     ["What is the size of bovine coronavirus?", """We sequenced the first Bovine coronavirus (BCoV) complete genome sequence from France. This BCoV was directly sequenced from a fecal sample collected from a calf in Normandy in 2014. B ovine coronavirus (BCoV) belongs to the Nidovirales order, the Coronaviridae family, the Coronavirinae subfamily, and the Betacoronavirus. Its genome is a single-stranded, linear, and nonsegmented RNA of around 31 kb. BCoV is responsible for respiratory and enteric diseases in cattle, particularly during winter. To date, the 19 complete BCoV genome sequences available in GenBank databases (consulted on 17 January 2017) originated from the United States or Asia. Here, we report the first complete genome sequence of a BCoV detected in France."""],
#     ["How many cysteine residues are contained in the first transmembrane domain of IFITM3?", """Recently, one of the interferon-induced transmembrane (IFITM) family proteins, IFITM3, has become an important target for the activity against influenza A (H1N1) virus infection. In this protein, a post-translational modification by fatty acids covalently attached to cysteine, termed S-palmitoylation, plays a crucial role for the antiviral activity. IFITM3 possesses three cysteine residues for the S-palmitoylation in the first transmembrane (TM1) domain and in the cytoplasmic (CP) loop. Because these cysteines are well conserved in the mammalian IFITM family proteins, the S-palmitoylation on these cysteines is significant for their functions. IFITM5 is another IFITM family protein and interacts with the FK506-binding protein 11 (FKBP11) to form a higher-order complex in osteoblast cells, which induces the expression of immunologically relevant genes."""],
#     ["What kinds of viruses are Japanese encephalitis virus(JEV), tick-borne encephalitis virus(TBEV), eastern equine encephalitis virus (EEEV), sindbis virus(SV), and dengue virus(DV)?","""The developed ELISA-array proved to have similar specificity and higher sensitivity compared with the conventional ELISAs. This method was validated by different viral cultures and three chicken eggs inoculated with infected patient serum. The results demonstrated that the developed ELISA-array is sensitive and easy to use, which would have potential for clinical use. Japanese encephalitis virus(JEV), tick-borne encephalitis virus(TBEV), eastern equine encephalitis virus (EEEV), sindbis virus(SV), and dengue virus(DV) are arboviruses. Establishment of an accurate and easy method for detection of these viruses is essential for the prevention and treatment of associated infectious diseases. """],
#     ["What method is useful in administering small molecules for systemic delivery to the body?", """In many studies, in vivo success has been demonstrated in delivering siRNAs to the lungs intranasally . An experimental setup of intranasal delivery by spray or droplet is simple and painless for the animal. Although the success in delivering siRNAs intranasally in rodents cannot be completely extrapolated to human use because of the significant differences in lung anatomy , this approach has potential for the clinical application of siRNAs. Phase II clinical trials have been initiated for the treatment of respiratory syncytial virus (RSV) infection, making use of intranasal application of naked chemically modified siRNA molecules that target viral gene products. Intranasal entry has long been used to administer small molecules, such as proteins, for systemic delivery. Because the nasal mucosa is highly vascularized, delivery of a thin epithelium of medication across the surface area can result in rapid absorption of the medication into the blood. Therefore, siRNAs administered intranasally might be deposited in the nose, and some of them may be unable to reach the lower respiratory tract. In fact, it has been reported that intranasal application of unformulated siRNAs resulted in lower delivery efficiency and homogeneous pulmonary distribution than that achieved with intratracheal application."""],
#     ["When did the last Director General of the WHO resign?", """In this month 2019s editorial, the PLOS Medicine Editors propose ideal qualities for the World Health Organization's next Director General, for whom the selection process is now underway. Response to the Ebola outbreak. Reformation of WHO to ready it to lead responses to future health emergencies is one area of active debate.Chan will step down from WHO on June 30, 2017 after more than a decade in the post. The process for choosing WHO's next leader has begun, promising to be protracted and rigorous as befits the importance of the role. Factoring in the many influential stakeholders in the process of appointing Chan's successor, however, transparency of the selection process may be one area unlikely to attract plaudits. Although too soon to speculate about the identity of WHO's next Director-General, it is worth reflecting on what qualities an incoming leader should bring to WHO and how that person might need to conceive changes in the structure and behavior of the organization against a landscape of important and evolving threats to the health of the fastgrowing global population. """],
#     ["What is the median incubation period?", "The incubation period is around 5 days (range: 4-7 days) with a maximum of 12-13 day"],
#     ["What is the incubation period range?", "The incubation period is around 5 days (range: 4-7 days) with a maximum of 12-13 day"],
#     ["What type of surfaces does it persist?", "The virus can survive on surfaces for up to 72 hours such as plastic and stainless steel ."]
# ]
sample_texts = [
    ["What is the conclusion?", "The conclusion is that there was a lag in the period of most intense 2009 pandemic influenza activity from southern to northern regions of Chile, significantly associated with geographical differences in maximum temperature and specific humidity. The latitudinal gradient in timing of pandemic activity was accompanied by a gradient in reproduction number (P < 0.0001). These two findings suggest that meteorological conditions may have modulated the transmissibility of 2009 A/H1N1 influenza in Chile. Our results could have significant public health implications for pandemic influenza control. Specifically, intensified surveillance strategies in southern regions could lead to early detection of pandemic influenza viruses and improved control outcomes. Furthermore, the spatial differences in the timing of local pandemic influenza in our study demonstrate the advantages of using high-resolution data to detect heterogeneous pandemic patterns. More studies are needed to determine whether these findings may be generalized to seasonal influenza epidemics and may be integrated in pandemic preparedness scenarios."],
    ["What is the common cause of upper and lower respiratory tract infections.?", df.select("body_text").collect()[4].body_text]
]
qa_df = spark.createDataFrame(sample_texts).toDF("question", "context")

In [48]:
model = pipeline.fit(qa_df).transform(qa_df)

In [49]:
model.select("question", "answer.result").show(truncate=False)

+--------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|question                                                                  |result                                                                                                                                                                                     |
+--------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|What is the conclusion?                                                   |[]                                                                                                                               